# Example: Monarch-KG notebook

Illustrates use of LinkML-Store over the Monarch-KG database (duckdb serialization)

First we initialize a `Client` object:

In [4]:
from linkml_store.api.client import Client

client = Client()

Next we download the dump (using pystow, to cache if needed)

In [5]:
from linkml_store.constants import LINKML_STORE_MODULE

MONARCH_KG_DB = "https://data.monarchinitiative.org/monarch-kg/latest/monarch-kg.db.gz"

path = LINKML_STORE_MODULE.ensure_gunzip(url=MONARCH_KG_DB, autoclean=True)

In [6]:
database = client.attach_database(f"duckdb:///{path}", "monarch-kg")

In [7]:
edges_coll = database.get_collection("denormalized_edges")

/Users/cjm/Library/Caches/pypoetry/virtualenvs/linkml-store-8ZYO4kTy-py3.10/lib/python3.10/site-packages/duckdb_engine/__init__.py:181: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [8]:
qr = edges_coll.find()
qr.rows_dataframe

ProgrammingError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name tables does not exist!
Did you mean "temp.information_schema.tables"?
LINE 2: FROM information_schema.tables
             ^
[SQL: SELECT COUNT(*)
FROM information_schema.tables
WHERE table_name = 'denormalized_edges']
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
key = ("subject_category", "object_category")
facets = edges_coll.query_facets(facet_columns=[key])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def heatmap(data):
    
    # Convert data into a DataFrame
    df = pd.DataFrame(data, columns=['Source', 'Target', 'Value'])
    
    # Pivot the data to create a matrix suitable for a heatmap
    pivot_df = df.pivot(index='Source', columns='Target', values='Value').fillna(0)

    
    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(pivot_df, annot=True, fmt=".0f", cmap="YlGnBu", cbar_kws={'label': 'Frequency'})
    plt.title("Heatmap of Biolink Data Frequencies")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    plt.show()


In [ ]:
heatmap(facets[key])